In [1]:
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer,util

In [3]:
import torch

In [4]:
#getting language model for symmetric semantic search(rephrase same question)
model=SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

C:\Users\mmanasanjani\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mmanasanjani\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#building corpus(a sentence)
#extracting sbert corpus from github https://github.com/laxmimerit
import requests

In [6]:
response=requests.get("https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/master/text-dataset-for-machine-learning/sbert-corpus.txt")
corpus=response.text.split('\r\n')
corpus

['A man is eating food.',
 'A man is eating a piece of bread.',
 'A man is eating pasta.',
 'The girl is carrying a baby.',
 'The baby is carried by the woman',
 'A man is riding a horse.',
 'A man is riding a white horse on an enclosed ground.',
 'A monkey is playing drums.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah is running behind its prey.',
 'A cheetah chases prey on across a field.']

In [7]:
responseq=requests.get("https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/master/text-dataset-for-machine-learning/sbert-queries.txt")
queries=responseq.text.split('\r\n')

In [8]:
queries

['A man is eating pasta.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah chases prey on across a field.']

In [9]:
#getting matching question(symmetric semantic search) similar to google
corpus_embeddings=model.encode(corpus,convert_to_tensor=True)
queries_embeddings=model.encode(queries,convert_to_tensor=True)

In [10]:
#let's normalize vectors for fast calculations
#cosine similarity between multiple vectors(corpus)
#instead of whole vector space we are going to do dot product
#for that, we should normalize vector
corpus_embeddings[0]

tensor([ 5.6192e-02,  1.8235e-02,  2.3670e-02, -9.4500e-03, -8.8719e-02,
        -7.9260e-03,  1.2745e-01, -5.0578e-02, -4.5912e-02, -3.8198e-02,
         5.0214e-02, -1.1839e-01, -6.2974e-02,  1.0486e-02,  1.3573e-03,
        -1.1340e-01,  3.3087e-02, -8.4886e-02,  8.9617e-02,  8.3860e-03,
         3.9334e-02,  6.8932e-02,  3.8834e-02, -1.0982e-01, -4.0440e-02,
        -2.5900e-02,  2.4625e-02, -5.3700e-02, -6.7910e-03,  4.6003e-02,
         1.0470e-01, -9.7853e-03,  1.6974e-02, -5.5376e-02, -2.8779e-02,
        -1.6152e-02,  2.3247e-02, -3.6139e-02, -5.9265e-03,  1.6874e-02,
        -5.4818e-02, -5.0854e-02, -1.4955e-02,  5.4211e-03,  1.2918e-03,
         6.1188e-02, -9.7889e-02,  8.7380e-03,  5.4717e-02, -3.3443e-02,
        -1.3637e-01,  1.7047e-02, -1.4716e-02, -7.7342e-03,  6.1286e-02,
        -2.3434e-02,  3.1187e-02,  3.2096e-02, -4.1245e-02, -2.8374e-02,
         7.7506e-02,  3.8242e-03,  3.6766e-02,  6.7190e-02,  1.2711e-01,
        -2.2905e-02,  6.4229e-03, -2.7661e-02, -2.2

In [11]:
corpus_embeddings=util.normalize_embeddings(corpus_embeddings)
queries_embeddings=util.normalize_embeddings(queries_embeddings)

In [12]:
len(corpus_embeddings[0])

384

In [13]:
hits=util.semantic_search(queries_embeddings,corpus_embeddings,score_function=util.dot_score) #semantic search will do matchings of queries embeddings with corpus embeddings

In [14]:
hits

[[{'corpus_id': 2, 'score': 0.9999998211860657},
  {'corpus_id': 0, 'score': 0.8384666442871094},
  {'corpus_id': 1, 'score': 0.7468274831771851},
  {'corpus_id': 5, 'score': 0.5328126549720764},
  {'corpus_id': 6, 'score': 0.36956483125686646},
  {'corpus_id': 7, 'score': 0.358096182346344},
  {'corpus_id': 3, 'score': 0.3572564721107483},
  {'corpus_id': 10, 'score': 0.2998370826244354},
  {'corpus_id': 9, 'score': 0.2915250062942505},
  {'corpus_id': 4, 'score': 0.26753929257392883}],
 [{'corpus_id': 8, 'score': 1.0},
  {'corpus_id': 7, 'score': 0.7612734436988831},
  {'corpus_id': 3, 'score': 0.3815288245677948},
  {'corpus_id': 6, 'score': 0.36845868825912476},
  {'corpus_id': 10, 'score': 0.3676949739456177},
  {'corpus_id': 5, 'score': 0.3185703456401825},
  {'corpus_id': 9, 'score': 0.31449681520462036},
  {'corpus_id': 1, 'score': 0.27469491958618164},
  {'corpus_id': 0, 'score': 0.2099388688802719},
  {'corpus_id': 2, 'score': 0.2016819268465042}],
 [{'corpus_id': 10, 'score'

In [16]:
top_hits=util.semantic_search(queries_embeddings,corpus_embeddings,score_function=util.dot_score,top_k=3) #semantic search will do matchings of queries embeddings with corpus embeddings
#topk gives 3 matching results of each of the search query

In [17]:
top_hits

[[{'corpus_id': 2, 'score': 0.9999998211860657},
  {'corpus_id': 0, 'score': 0.8384666442871094},
  {'corpus_id': 1, 'score': 0.7468274831771851}],
 [{'corpus_id': 8, 'score': 1.0},
  {'corpus_id': 7, 'score': 0.7612734436988831},
  {'corpus_id': 3, 'score': 0.3815288245677948}],
 [{'corpus_id': 10, 'score': 1.0000001192092896},
  {'corpus_id': 9, 'score': 0.870399534702301},
  {'corpus_id': 6, 'score': 0.37411707639694214}]]

In [19]:
#validating the search
queries

['A man is eating pasta.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah chases prey on across a field.']

In [20]:
corpus

['A man is eating food.',
 'A man is eating a piece of bread.',
 'A man is eating pasta.',
 'The girl is carrying a baby.',
 'The baby is carried by the woman',
 'A man is riding a horse.',
 'A man is riding a white horse on an enclosed ground.',
 'A monkey is playing drums.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah is running behind its prey.',
 'A cheetah chases prey on across a field.']

In [22]:
for query,hit in zip(queries,top_hits): #iterate queries and hits together, total 3 search queries, top 3 corpus, so 3 vectors
    for q_hit in hit:
        id=q_hit['corpus_id']
        score=q_hit['score']
        
        print(query,"<>",corpus[id],'-->',score)
    
    print()

A man is eating pasta. <> A man is eating pasta. --> 0.9999998211860657
A man is eating pasta. <> A man is eating food. --> 0.8384666442871094
A man is eating pasta. <> A man is eating a piece of bread. --> 0.7468274831771851

Someone in a gorilla costume is playing a set of drums. <> Someone in a gorilla costume is playing a set of drums. --> 1.0
Someone in a gorilla costume is playing a set of drums. <> A monkey is playing drums. --> 0.7612734436988831
Someone in a gorilla costume is playing a set of drums. <> The girl is carrying a baby. --> 0.3815288245677948

A cheetah chases prey on across a field. <> A cheetah chases prey on across a field. --> 1.0000001192092896
A cheetah chases prey on across a field. <> A cheetah is running behind its prey. --> 0.870399534702301
A cheetah chases prey on across a field. <> A man is riding a white horse on an enclosed ground. --> 0.37411707639694214

